In [11]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Model
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, Trainer
import torch.nn.functional as F
from torch.utils.data import DataLoader
from datasets import Dataset, DatasetDict
from torch.optim import AdamW
import torch
import json
from datasets import load_dataset
from pprint import pprint
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [12]:
def load_data(filepath):
    with open(filepath) as json_file:
        return json.load(json_file)

train_data = load_data("./spider_data/train_spider.json")
train_tables_data = load_data("./spider_data/tables.json")

test_data = load_data("./spider_data/test.json")
test_tables_data = load_data("./spider_data/test_tables.json")


In [13]:
def generate_table_index(tables):
    index_dictionary = {}
    for index, table in enumerate(tables):
        index_dictionary[table['db_id']] = index
    return index_dictionary

train_table_index_dictionary = generate_table_index(train_tables_data)
test_table_index_dictionary = generate_table_index(test_tables_data)


In [14]:

def parse_table(db_id, tables, table_index_dictionary):
    table = tables[table_index_dictionary[db_id]]
    table_names = []
    column = {}
    # print(table['column_names_original'])
    for table_name in table['table_names_original']:
        table_names.append(table_name)
        column[table_name] = []
    for index, column_name in table['column_names_original']:
        if(index == -1): 
            continue
        column[table_names[index]].append(column_name)
    return column

def parse(entry, tables, table_index_dictionary):
    question = entry['question']
    target = entry['query']
    db_id = entry['db_id']
    schema = parse_table(db_id, tables, table_index_dictionary)
    # print(schema, question, target)
    return {'input': f"Given the following SQL Schema: {schema}. Provide a SQL query reponse for: {question}", 'target': target}

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")

def preprocess_data(examples):
    inputs = [item for item in examples['input']]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding=True, return_tensors="pt")
    
    targets = [item for item in examples['target']]
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=521, truncation=True, padding=True, return_tensors="pt")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [15]:

raw_training_data = [parse(train_data[i], train_tables_data, train_table_index_dictionary) for i in range(len(train_data))]
pprint(raw_training_data[0])
raw_testing_data = [parse(test_data[i], test_tables_data, test_table_index_dictionary) for i in range(len(test_data))]
training_dataset = Dataset.from_list(raw_training_data)
testing_dataset = Dataset.from_list(raw_testing_data)
dataset_dict = DatasetDict({
    "train": training_dataset,
    "test": testing_dataset
})
encoded_dataset = dataset_dict.map(preprocess_data, batched=True)
encoded_dataset = encoded_dataset.remove_columns(['input', 'target'])
pprint(encoded_dataset)



{'input': "Given the following SQL Schema: {'department': ['Department_ID', "
          "'Name', 'Creation', 'Ranking', 'Budget_in_Billions', "
          "'Num_Employees'], 'head': ['head_ID', 'name', 'born_state', 'age'], "
          "'management': ['department_ID', 'head_ID', 'temporary_acting']}. "
          'Provide a SQL query reponse for: How many heads of the departments '
          'are older than 56 ?',
 'target': 'SELECT count(*) FROM head WHERE age  >  56'}


Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

/home/bryan/Projects/Text2Sql/cuda/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2147 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 7000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2147
    })
})


In [16]:
def query(db_id, query, Test=False):
    if Test == False:
        db_directory = './spider_data/database'
    else:
        db_directory = './spider_data/test_database'
    con = sqlite3.connect(f"{db_directory}/{db_id}/{db_id}.sqlite")
    cur = con.cursor()
    try:
        cur.execute(query)
        res = cur.fetchall()
    except:
        res = -1
    finally:
        con.close()
        return res


In [17]:
def custom_collate_fn(batch):
    # Separate metadata from model inputs
    input_ids = [item["input_ids"] for item in batch]
    attention_mask = [item["attention_mask"] for item in batch]
    labels = [item["labels"] for item in batch]
    # metadata = [item["metadata"] for item in batch]  # Collect metadata separately

    # Convert to tensors for model input
    return {
        "input_ids": torch.tensor(input_ids),
        "attention_mask": torch.tensor(attention_mask),
        "labels": torch.tensor(labels),
        # "metadata": metadata,  # Keep metadata for later use
    }

In [18]:
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small")
model.to(device)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
train_loader = DataLoader(encoded_dataset['train'], batch_size=8, shuffle=True, collate_fn=data_collator)
eval_loader = DataLoader(encoded_dataset['test'], batch_size=8, shuffle=True, collate_fn=data_collator)
optimizer = AdamW(model.parameters(), lr=5e-5)


In [ ]:

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    save_strategy="epoch",
    save_total_limit=2,
    predict_with_generate=True,
    remove_unused_columns=True,
    fp16=True,
)

# Define trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['test'],
    processing_class=tokenizer,
    data_collator=data_collator
    # compute_loss_func=custom_loss
)


# # Train the model
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss


In [ ]:
# Save the model
model.save_pretrained("./fine_tuned_t5_sql")
tokenizer.save_pretrained("./fine_tuned_t5_sql")